Source: https://tts.readthedocs.io/en/latest/tutorial_for_nervous_beginners.html#training-a-tts-model 

https://github.com/neonsecret/TTS-With-Voice-Cloning-Multilang

In [ ]:
!pip install TTS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.1/604.1 KB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 KB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.3/577.3 KB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 KB 24.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 KB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 KB 35.1 MB/s eta 0:00:00
     ━━━━

In [ ]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d freezerainml/ruslan

100% 3.84G/3.84G [03:24<00:00, 23.1MB/s]
100% 3.84G/3.84G [03:24<00:00, 20.2MB/s]


In [ ]:
!unzip '/content/ruslan.zip'

In [1]:
import os
from glob import glob

from trainer import Trainer, TrainerArgs

from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.vits import CharactersConfig, Vits, VitsArgs, VitsAudioConfig
from TTS.tts.utils.languages import LanguageManager
from TTS.tts.utils.speakers import SpeakerManager
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

output_path = "/content/"

dataset_config = BaseDatasetConfig(
    formatter="ruslan", meta_file_train="/content/ruslan_dataset/metadata_RUSLAN_22200.csv", 
    path='/content/ruslan_dataset/'
)

audio_config = VitsAudioConfig(
    sample_rate=16000,
    win_length=1024,
    hop_length=256,
    num_mels=80,
    mel_fmin=0,
    mel_fmax=None,
)

vitsArgs = VitsArgs(
    use_language_embedding=True,
    embedded_language_dim=4,
    use_speaker_embedding=True,
    use_sdp=False,
)
config = VitsConfig(
    model_args=vitsArgs,
    audio=audio_config,
    run_name="vits_vctk",
    use_speaker_embedding=True,
    save_all_best=True,
    batch_size=32,
    eval_batch_size=16,
    batch_group_size=0,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=100,
    text_cleaner="multilingual_cleaners",
    use_phonemes=False,
    phoneme_language="ru-ru",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    compute_input_seq_cache=True,
    print_step=25,
    use_language_weighted_sampler=True,
    print_eval=False,
    mixed_precision=False,
    min_audio_len=32 * 256 * 4,
    max_audio_len=160000,
    output_path=output_path,
    datasets=[dataset_config],
    characters=CharactersConfig(
        characters_class="TTS.tts.models.vits.VitsCharacters",
        pad="<PAD>",
        eos="<EOS>",
        bos="<BOS>",
        blank="<BLNK>",
        characters="!¡'(),-.:;¿?abcdefghijklmnopqrstuvwxyzµßàáâäåæçèéêëìíîïñòóôöùúûüąćęłńœśşźżƒабвгдежзийклмнопрстуфхцчшщъыьэюяёєіїґӧ «°±µ»$%&‘’‚“`”„",
        punctuations="!¡'(),-.:;¿? ",
        phonemes=None,
    ),
    test_sentences=[
        [
            "It took me quite a long time to develop a voice, and now that I have it I'm not going to be silent.",
            "mary_ann",
            None,
            "en_US",
        ],
        [
            "Il m'a fallu beaucoup de temps pour d\u00e9velopper une voix, et maintenant que je l'ai, je ne vais pas me taire.",
            "ezwa",
            None,
            "fr_FR",
        ],
        ["Ich finde, dieses Startup ist wirklich unglaublich.", "eva_k", None, "de_DE"],
        ["Я думаю, что этот стартап действительно удивительный.", "oblomov", None, "ru_RU"],
    ],
)

# init audio processor
ap = AudioProcessor.init_from_config(config)

# load training samples
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)


# INITIALIZE THE TOKENIZER
# Tokenizer is used to convert text to sequences of token IDs.
# config is updated with the default characters if not defined in the config.
tokenizer, config = TTSTokenizer.init_from_config(config)

# init model
model = Vits(config, ap, tokenizer)

# init the trainer and 🚀
trainer = Trainer(
    TrainerArgs(),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)

 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 | > Found 22200 files in /content/ruslan_dataset


 > Training Environment:
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 2
 | > Num. of Torch Threads: 1
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 > Start Tensorboard: tensorboard --logdir=/content/vits_vctk-March-05-2023_07+59PM-0000000

 > Model has 82090365 parameters


In [2]:
trainer.fit()


 > EPOCH: 0/100
 --> /content/vits_vctk-March-05-2023_07+59PM-0000000




> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 21978


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(

 > TRAINING (2023-03-05 19:59:55) 


 | > Preprocessing samples
 | > Max text length: 132
 | > Min text length: 15
 | > Avg text length: 77.22354371498044
 | 
 | > Max audio length: 159989.0
 | > Min audio length: 32788.0
 | > Avg audio length: 103850.23897875039
 | > Num. instances discarded samples: 3060
 | > Batch group size: 0.
['<BLNK>', '–', '<BLNK>', ' ', '<BLNK>', 'ч', '<BLNK>', 'т', '<BLNK>', 'о', '<BLNK>', ' ', '<BLNK>', 'ж', '<BLNK>', 'е', '<BLNK>', ' ', '<BLNK>', 'т', '<BLNK>', 'ы', '<BLNK>', ' ', '<BLNK>', 'п', '<BLNK>', 'р', '<BLNK>', 'а', '<BLNK>', 'з', '<BLNK>', 'д', '<BLNK>', 'н', '<BLNK>', 'о', '<BLNK>', 'в', '<BLNK>', 'а', '<BLNK>', 'л', '<BLNK>', ',', '<BLNK>', ' ', '<BLNK>', 'с', '<BLNK>', 'ы', '<BLNK>', 'н', '<BLNK>', 'о', '<BLNK>', 'к', '<BLNK>', '?', '<BLNK>']
 [!] Character '–' not found in the vocabulary. Discarding it.
['<BLNK>', 'ж', '<BLNK>', 'и', '<BLNK>', 'т', '<BLNK>', 'ь', '<BLNK>', ' ', '<BLNK>', 'п', '<BLNK>', 'р', '<BLNK>', 'е', '<BLNK>', 'д', '<BLNK>', 'п', '<BLNK>', 'о', '<BLNK>', 'ч'

/usr/local/lib/python3.8/dist-packages/torch/functional.py:632: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:801.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


...

 ! Run is removed from /content/vits_vctk-March-05-2023_07+59PM-0000000


 | > Synthesizing test sentences.


Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/trainer/trainer.py", line 1591, in fit
    self._fit()
  File "/usr/local/lib/python3.8/dist-packages/trainer/trainer.py", line 1548, in _fit
    self.test_run()
  File "/usr/local/lib/python3.8/dist-packages/trainer/trainer.py", line 1466, in test_run
    test_outputs = self.model.test_run(self.training_assets)
  File "/usr/local/lib/python3.8/dist-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/TTS/tts/models/vits.py", line 1438, in test_run
    aux_inputs = self.get_aux_input_from_test_sentences(s_info)
  File "/usr/local/lib/python3.8/dist-packages/TTS/tts/models/vits.py", line 1409, in get_aux_input_from_test_sentences
    speaker_id = self.speaker_manager.name_to_id[speaker_name]
AttributeError: 'NoneType' object has no attribute 'name_to_id'


SystemExit: ignored

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# cleanup "output" from cache
import shutil
shutil.rmtree("./phoneme_cache")